In [ ]:
**Implementation of Approximate Stream Regression**


While recieving stream data we need to continously update the value of parameters as we recieve it. $X_k$ is the data that is recieved in the $k^{th}$ window.


$\beta_k ^{'} = (X_k^TX_k)^{-1}X^T_ky_k$

$\beta_k = (1-\alpha)\beta_k ^{'}+ (\alpha)\beta_{k-1}$

Here $\alpha$ is Smoothing factor \\
$\beta_k^{'}$ is Parameter vector for the kth window calculated considering only the data records of kth window. \\

$\beta_{k-1}$ is Parameter vector for the kth window calculated considering all data records seen upto (k-1)th window. \\

$\beta_{k}$ is Parameter vector for the kth window calculated considering all data records seen upto (k)th window. \\

**Importing Libaries**

from sklearn import metrics
import pandas as pd

**Loading the dataset and splitting the test set**

count=0

testdataframe=pd.read_csv("boston.csv",header=None)
testdataarray=testdataframe.values
X_test=testdataarray[400:506,:-1] 
y_test=testdataarray[400:506,-1]
#print(y_test)

**Simulation of Streaming Data**

**Function for generating stream data**


**Description : This function generates batches of data with size equal to args:(wsize) and instead of using return uses yield which will**

**enable to generate next batch of training data each time it is iterated**

args : wsize

yield : stream data

def myGeneratorfun(wsize):
    print("In myGeneratorfun:")
    counter=0
    for chunk in pd.read_csv('boston.csv', header=None, chunksize=wsize):
        print("In for loop, counter vlaue is", counter)
        chunk_array=chunk.values
        counter=counter+1
        yield (chunk_array[:,:-1], chunk_array[:,-1])
        if counter >= 8:
            break

**Calling stream data genrator with wsize = 50**

myGenerator = myGeneratorfun(50)

print(X_test.shape)

**Steps for ASR**


M=np.zeros((X_test.shape[1],X_test.shape[1]))

V=np.zeros((X_test.shape[1]))
Beta=np.zeros(X_test.shape[1])

alpha = 0.75
Beta_k = Beta

flag = 0

for (x,y) in myGenerator:

    try:
        XTXi=inv(np.dot(x.T,x))
    except LinAlgError:
        XTXi=pinv(np.dot(x.T,x))

    XTy=np.dot(x.T,y)

    Beta_k_dash=np.dot(XTXi,XTy)

    if flag==0 :
        Beta_k = Beta_k_dash
        flag=1
    else :
        Beta_k = (1-alpha)*Beta_k_dash + alpha*Beta_k

    print("Beta':", Beta_k_dash)
    print("Betas:", Beta_k)
    print("Betas Shape:", Beta.shape)   

    predictions=np.dot(Beta_k,X_test.T)
    #print(predictions.shape)
    print("MAE:", metrics.mean_absolute_error(y_true=y_test,y_pred=predictions))
    print("MSE:", metrics.mean_squared_error(y_true=y_test,y_pred=predictions))

M.shape